In [ ]:
import numpy as np
import cv2
from tqdm import trange
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(); sns.set_style('dark')

In [ ]:

## Functions
def rgb(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

def bgr(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

def gray(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def uint8ify(img):
    img -= img.min()
    img /= img.max()
    img *= 255
    return np.uint8(img)

def overlay(a,b):
    # a and b should be float images and between 0-1

    mask = a >= 0.5 # generate boolean mask of everywhere a > 0.5 
    ab = np.zeros_like(a) # generate an output container for the blended image 

    # now do the blending 
    ab[~mask] = (2*a*b)[~mask] # 2ab everywhere a<0.5
    ab[mask] = (1-2*(1-a)*(1-b))[mask] # else this
    
    return ab

def disp(img, title='', s=8, vmin=None, vmax=None, write=False, file_name=None):
    plt.figure(figsize=(s,s))
    plt.axis('off')
    if vmin is not None and vmax is not None:
        plt.imshow(img, cmap='gray', vmin=vmin, vmax=vmax)
    else:
        plt.imshow(img, cmap='gray')
    plt.title(title)
    if write and file_name is not None:
        plt.savefig(file_name)
    plt.show()
    
def before_after(img_a, img_b, name="", vmin=None, vmax=None, effect_name="Processed"):
    fig, axs = plt.subplots(1,2, constrained_layout=True, figsize=(10,4))
    axs[0].axis('off')
    axs[1].axis('off')
    axs[0].set_title(f"{name} Original")
    axs[1].set_title(f"{name} {effect_name}")
    if vmin is not None and vmax is not None:
        axs[0].imshow(img_a, cmap='gray', vmin=vmin, vmax=vmax)
        axs[1].imshow(img_b, cmap='gray', vmin=vmin, vmax=vmax)
    else:
        axs[0].imshow(img_a, cmap='gray')
        axs[1].imshow(img_b, cmap='gray')
    plt.show()


## Load Images

In [ ]:
img_1 = (cv2.imread("cart_1.png", 0))
img_1_rgb = rgb(cv2.imread("cart_1.png"))
img_2 = (cv2.imread("cart_2.png", 0))
img_2_rgb = rgb(cv2.imread("cart_2.png"))
items = []
for i in range(12):
    items.append(cv2.imread(f"items/{i+1}.png", 0))
    disp(items[i], s=2)

In [ ]:
names = [
    "Tuna",
    "Peril",
    "Pepsi",
    "Milk",
    "Tiny",
    "Crest",
    "Cake",
    "Oil",
    "Maxi",
    "Shampoo",
    "Tea",
    "Chips"
]

## Detect Features

In [ ]:

def detect_features(img, nfeatures=500, blur_size=0, edgeThreshold = 31):
    if blur_size>0:
        img = cv2.GaussianBlur(img, (blur_size,blur_size), None)
    # Initiate ORB detector
    orb = cv2.ORB_create(nfeatures=nfeatures, edgeThreshold=edgeThreshold)
    # find the keypoints with ORB
    kp = orb.detect(img,None, )
    # compute the descriptors with ORB
    kp, des = orb.compute(img, kp)
    # draw only keypoints location,not size and orientation
    drawn = cv2.drawKeypoints(img, kp, None, color=(255,0,0), flags=0)

    return kp, des, drawn

### BG No Blur

In [ ]:
kp_img_1, des_img_1, drawn_img_1 = detect_features(img_1, nfeatures=100000)
disp(drawn_img_1,"cart_1 Features",s=13)

kp_img_2, des_img_2, drawn_img_2 = detect_features(img_2, nfeatures=100000)
disp(drawn_img_2,"cart_2 Features",s=13)

### BG Blur

In [ ]:
kp_img_1, des_img_1, drawn_img_1 = detect_features(img_1, nfeatures=13000, blur_size=3, edgeThreshold=31)
disp(drawn_img_1,"cart_1 Features",s=13)

kp_img_2, des_img_2, drawn_img_2 = detect_features(img_2, nfeatures=13000, blur_size=3)
disp(drawn_img_2,"cart_2 Features",s=13)

### Items

In [ ]:
items_kp = []
items_des = []
for i in range(len(items)):
    kp, des, drawn = detect_features(items[i], nfeatures=300)
    items_kp.append(kp)
    items_des.append(des)
    disp(drawn,f"item_{i} Features",s=5)

## Matching

In [ ]:
def match_features(img_a, img_b, des_a, des_b, kp_a, kp_b, n_matches_to_show=10, ):
    # create BFMatcher object
    bf = cv2.BFMatcher(cv2.NORM_HAMMING2, crossCheck=True)
    # Match descriptors.
    matches = bf.match(des_a,des_b)
    # Sort them in the order of their distance.
    matches = sorted(matches, key = lambda x:x.distance)
    drawn = cv2.drawMatches(img_a, kp_a, img_b, kp_b, matches[:n_matches_to_show],None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    
    return matches, drawn

In [ ]:
for i in trange(len(items)):
    matches, drawn = match_features(
        img_1, items[i],
        des_img_1, items_des[i],
        kp_img_1, items_kp[i],
        n_matches_to_show=50)

    disp(drawn, s=15)


In [ ]:
for i in trange(len(items)):
    matches, drawn = match_features(
        img_2, items[i],
        des_img_2, items_des[i],
        kp_img_2, items_kp[i],
        n_matches_to_show=50)

    disp(drawn, s=15)


## ORB 2 + homography

In [ ]:
def do_hom(img_a, img_b, des1, des2, kp1, kp2, debug=True, min_match_count=100):
    img_a = img_a.copy()
    img_a_rgb = np.repeat(img_a[:,:,np.newaxis], 3, axis=2)
    img_b_rgb = np.repeat(img_b[:,:,np.newaxis], 3, axis=2)
    MIN_MATCH_COUNT = min_match_count
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks = 50)
    des1 = np.float32(des1)
    des2 = np.float32(des2)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1,des2,k=2)

    # store all the good matches as per Lowe's ratio test.
    good = []
    for m,n in matches:
        if m.distance < 0.7*n.distance:
            good.append(m)

    if len(good)>MIN_MATCH_COUNT:
        src_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
        dst_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
        matchesMask = mask.ravel().tolist()
        h,w = img_b.shape
        pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
        dst = cv2.perspectiveTransform(pts,M)
        # img_a_rgb = cv2.polylines(img_a_rgb,[np.int32(dst)],True,255,3, cv2.LINE_AA)
        img_a = cv2.fillPoly(img_a,[np.int32(dst)],0)
        print( "n_features - {}/{}".format(len(good), MIN_MATCH_COUNT) )

        if debug:
            # disp(img_a, s=12)
            pass
    else:
        print( "Not enough matches are found - {}/{}".format(len(good), MIN_MATCH_COUNT) )
        matchesMask = None
        return False, img_a

    if debug:
        draw_params = dict(
        matchColor = (0,255,0), # draw matches in green color
        singlePointColor = None,
        matchesMask = matchesMask, # draw only inliers
        flags = 2)
        drawn = cv2.drawMatches(img_a_rgb,kp1,img_b_rgb,kp2,good,None,**draw_params)
        disp(drawn, s=12)
    return True, img_a

In [ ]:
def find_multiple(img_a, img_b, des1, des2, kp1, kp2, debug=True, min_match_count=100):
    img_a = img_a.copy()
    ret = True
    counter = 0
    while ret:
        ret, img_a = do_hom(img_a, img_b, des1, des2, kp1, kp2, debug=debug, min_match_count=min_match_count)
        kp1, des1, drawn = detect_features(img_a, nfeatures=13000, blur_size=3)
        if ret:
            counter+=1
            print(f"Counter: {counter}")
    return counter

#### Debug Basket 1

In [ ]:
thresholds = [
    100,
    100,
    44,
    100,
    100,
    100,
    100,
    80,
    100,
    100,
    100,
    100
]
basket_1 = []
for i in range(len(items)):
    print("Counting", names[i], "...")
    count = find_multiple(
        img_1,
        items[i],
        des_img_1,
        items_des[i],
        kp_img_1,
        items_kp[i], debug=True, min_match_count=thresholds[i])
    print("-"*40)
    print()
    basket_1.append(count)

disp(img_1_rgb, title="Basket_1", s=13)
print("Basket 1 Summary")
print("-"*15)
for i in range(len(basket_1)):
    if basket_1[i]>0:
        print(f"{names[i]}\t: x {basket_1[i]}")

#### No Debug Basket 1

In [ ]:
basket_1 = []
for i in range(len(items)):
    print("Counting", names[i])
    count = find_multiple(
        img_1,
        items[i],
        des_img_1,
        items_des[i],
        kp_img_1,
        items_kp[i], debug=True, min_match_count=thresholds[i])
    print("-"*40)
    print()
    basket_1.append(count)

disp(img_1_rgb, title="Basket_1", s=13)
print("Basket 1 Summary")
print("-"*15)
for i in range(len(basket_1)):
    if basket_1[i]>0:
        print(f"{names[i]}\t: x {basket_1[i]}")

#### Basket 2

In [ ]:
thresholds2 = [
    45,
    100,
    30,
    100,
    70,
    50,
    45,
    50,
    50,
    85,
    85,
    50
]

names = [
    "Tuna",
    "Peril",
    "Pepsi",
    "Milk",
    "Tiny",
    "Crest",
    "Cake",
    "Oil",
    "Maxi",
    "Shampoo",
    "Tea",
    "Chips"
]
basket_2 = []
for i in range(len(items)):
    print("Counting", names[i], "...")
    count = find_multiple(
        img_2,
        items[i],
        des_img_2,
        items_des[i],
        kp_img_2,
        items_kp[i], debug=True, min_match_count=thresholds2[i])
    print("-"*40)
    print()
    basket_2.append(count)

disp(img_2_rgb, title="Basket_2", s=13)
print("Basket 2 Summary")
print("-"*15)
for i in range(len(basket_2)):
    if basket_2[i]>0:
        print(f"{names[i]}\t: x {basket_2[i]}")

In [ ]:
thresholds2 = [
    45,
    100,
    30,
    100,
    70,
    50,
    45,
    50,
    50,
    85,
    85,
    50
]

names = [
    "Tuna",
    "Peril",
    "Pepsi",
    "Milk",
    "Tiny",
    "Crest",
    "Cake",
    "Oil",
    "Maxi",
    "Shampoo",
    "Tea",
    "Chips"
]
basket_2 = []
for i in range(len(items)):
    print("Counting", names[i], "...")
    count = find_multiple(
        img_2,
        items[i],
        des_img_2,
        items_des[i],
        kp_img_2,
        items_kp[i], debug=False, min_match_count=thresholds2[i])
    print("-"*40)
    print()
    basket_2.append(count)

disp(img_2_rgb, title="Basket_2", s=13)
print("Basket 2 Summary")
print("-"*15)
for i in range(len(basket_2)):
    if basket_2[i]>0:
        print(f"{names[i]}\t: x {basket_2[i]}")

## Template Matching

In [ ]:
def template_match(img_a, img_b, img_a_rgb, threshold=0.9, debug=True, name=None):
    # make a copy because we are going to change the content of this image
    if debug:
        img_display = img_a_rgb.copy()

    # size of the template
    h, w = img_b.shape[:2]

    # Template matching
    result = cv2.matchTemplate(img_a, img_b, cv2.TM_CCOEFF_NORMED)
    cv2.normalize( result, result, 0, 1, cv2.NORM_MINMAX, -1 )

    # Pad the result to match the size of img_a and make it rgb in order to draw colored rectangles on it
    if debug:
        result_padded = np.zeros_like(img_a)
        result_padded[img_b.shape[0]//2:img_b.shape[0]//2+result.shape[0],img_b.shape[1]//2:img_b.shape[1]//2+result.shape[1]] = result*255
        result_padded = np.repeat(result_padded[:, :, np.newaxis], 3, axis=2)
    
    # Only count those points with high enough matching score
    max_val = 1
    matched_points = []
    while max_val > threshold:
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
        matchLoc = max_loc
        
        if max_val > threshold:
            matched_points.append(matchLoc)

            # Delete the already matched pixels(to be able to match the other ones next time)
            result[max(matchLoc[1]-h//2, 0):min(matchLoc[1]+h//2, result.shape[0]), max(matchLoc[0]-w//2, 0):min(matchLoc[0]+w//2, result.shape[1])] = 0

            if debug:
                # Draw rectangle on the source rgb image
                cv2.rectangle(
                    img_display,
                    (matchLoc[0], matchLoc[1]),
                    (matchLoc[0] + w, matchLoc[1] + h),
                    (255,0,0), 8, 6, 0 )

                # Draw rectangle on the padded result image
                cv2.rectangle(
                    result_padded, 
                    (matchLoc[0], matchLoc[1]),
                    (matchLoc[0] + w, matchLoc[1] + h),
                    (0,0,255), 8, 6, 0 )

    if debug:
        if name is None:
            disp(img_display, s=13)
            disp(result_padded, s=13)
        else:
            disp(img_display, s=13, title=f"{name} Matched", write=False, file_name=f"item_{name}_Matched_plt.png")
            disp(result_padded, s=13, title=f"{name} Match Score", write=False, file_name=f"item_{name}_Match_score_plt.png")
            cv2.imwrite(f"item_{name}_Matched.png", bgr(img_display))
            cv2.imwrite(f"item_{name}_Match_score.png", bgr(result_padded))

    return matched_points

In [ ]:
matched_points = template_match(img_1, items[0], img_1_rgb, threshold=0.9, name=names[0])
print("matched_points:", len(matched_points))

### Image_1

In [ ]:
basket_1 = []
for i in trange(len(items)):
    matched_points = template_match(img_1, items[i], img_1_rgb, threshold=0.9, debug=False, name=names[i])
    basket_1.append(len(matched_points))

print("Basket 1 Summary")
print("-"*15)
for i in range(len(basket_1)):
    if basket_1[i]>0:
        print(f"{names[i]}\t: x {basket_1[i]}")


In [ ]:
basket_2 = []
for i in trange(len(items)):
    matched_points = template_match(img_2, items[i], img_2_rgb, threshold=0.9, debug=True, name=names[i])
    basket_2.append(len(matched_points))

print("Basket 2 Summary")
print("-"*15)
for i in range(len(basket_2)):
    if basket_2[i]>0:
        print(f"{names[i]}\t: x {basket_2[i]}")
